In [1]:
import os
import torchvision

torchvision.disable_beta_transforms_warning()

import json
import numpy as np
import random
import torch
import torch.nn as nn

if os.getcwd().split("/")[-1] == "notebooks":
    os.chdir("../..")

from inference.inference_model_avg import SFDInferenceModelAvg
from inference.image_generation.image_generator import ImageGenerator
from inference.utils import concatenate_2x2

In [59]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sdf_inference = SFDInferenceModelAvg("results/training_cigarettes_ouns")

Has found /embds_pooled.h5 files, will only use /embds_pooled.h5
has_pooled_embds set to True
Has found /embds.h5 files, will only use /embds_pooled.h5
has_embds set to True
Initializing Properties class...
property_to_pid :  {'A blond girl': 0, 'A brunette girl': 1, 'with blue eyes': 2, 'with brown eyes': 3, 'walking down the street': 4, 'walking in the rain': 5, 'at a market': 6, 'sitting at a cafe': 7, 'sitting in a bar': 8, 'on horseback in the forest': 9, 'on horseback across a plain': 10, 'at the beach': 11, 'on a boat': 12, 'driving a blue car': 13, 'driving a red car': 14, 'wearing a blue t-shit': 15, 'wearing a black t-shit': 16, 'wearing a red t-shit': 17, 'and a hat': 18, 'and a baseball cap': 19, 'holding a coffee': 20, 'holding a coca-cola': 21, 'holding a gun': 22, 'smoking a cigarette': 23, 'looking to the left': 24, 'looking to the right': 25, 'looking in front of her': 26, 'looking behind her': 27}
category_to_cid :  {'hair': 0, 'eyes': 1, 'situration': 2, 't_shirt': 3

In [60]:
sdf_inference.display_properties()

+-----------------+-----------------+-----------------------------+------------------------+--------------------+---------------------+-------------------------+
| hair            | eyes            | situration                  | t_shirt                | hat                | action              | pose                    |
+-----------------+-----------------+-----------------------------+------------------------+--------------------+---------------------+-------------------------+
| A blond girl    | with blue eyes  | walking down the street     | wearing a blue t-shit  | and a hat          | holding a coffee    | looking to the left     |
| A brunette girl | with brown eyes | walking in the rain         | wearing a black t-shit | and a baseball cap | holding a coca-cola | looking to the right    |
|                 |                 | at a market                 | wearing a red t-shit   |                    | holding a gun       | looking in front of her |
|                 |         

In [ ]:
idx1, prompt1 = sdf_inference.search_idx_prompt(
    [
        "A blond girl",
        "with brown eyes",
        "sitting in a bar",
        "wearing a blue t-shit",
        "and a baseball cap",
        "holding a gun",
        "looking to the left"
    ]
)



print("idx1 = ", idx1)
print("prompt1 = ", prompt1)

idx1 =  15
prompt1 =  A blond girl, with brown eyes, sitting in a bar, wearing a blue t-shit, and a baseball cap, holding a gun, looking to the left
idx2 =  17
prompt2 =  A blond girl, with brown eyes, walking down the street, wearing a red t-shit, and a baseball cap, holding a gun, looking in front of her


In [12]:
# load the diffusion model

image_generator = ImageGenerator(simulated=False, device=device)

Loading pipeline...


/home/ubuntu/miniconda3/envs/large_turbo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 2 files: 100%|██████████| 2/2 [00:00<00:00, 29746.84it/s]
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading pipeline components...: 100%|██████████| 9/9 [00:12<00:00,  1.38s/it]


Pipeline loaded.


In [ ]:
# generate original images

image_generator.generate_image_from_prompt(prompt=prompt1, image_name="image_blond_bar_holding_gun.png")

100%|██████████| 4/4 [00:06<00:00,  1.51s/it]


Image saved in image_blond_bar_holding_gun.png


100%|██████████| 4/4 [00:06<00:00,  1.50s/it]


Image saved in image_blond_street_holding_gun.png


In [ ]:
# generate a new embedding
embd1 = sdf_inference.get_x(idx=idx1)

In [ ]:
embd_1, pooled_embd_1 = sdf_inference.overwrite_full_embedding(embd1, idx=idx1)

In [16]:
print("embds = ", embd_1.shape)
print("pooled_embd_1 = ", pooled_embd_1.shape)

embds =  torch.Size([1, 333, 4096])
pooled_embd_1 =  torch.Size([1, 2048])


In [ ]:
image_generator.generate_image_from_embd(
   prompt_embeds=embd_1.detach(),
    pooled_prompt_embeds=pooled_embd_1.detach(),
    image_name="image_blond_bar_holding_gun_reconstructed.png",
)


100%|██████████| 4/4 [00:03<00:00,  1.19it/s]


Image saved in image_blond_bar_holding_gun_reconstructed.png


100%|██████████| 4/4 [00:03<00:00,  1.19it/s]


Image saved in image_blond_street_holding_gun_reconstructed.png


In [ ]:
custom_mask1 = sdf_inference.decoder.mask_arange[idx1 : idx1 + 1, :]
print(custom_mask1)

custom_mask1[0,21] = 0
custom_mask1[0,20] = 21


tensor([[ 0,  1,  0,  4,  0,  0,  0,  0,  9,  0,  0,  0,  0,  0,  0, 16,  0,  0,
          0, 20,  0, 22,  0,  0, 25,  0,  0,  0]])
tensor([[ 0,  1,  0,  4,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 18,
          0, 20,  0, 22,  0,  0,  0,  0, 27,  0]])
tensor(0)
tensor([[ 0,  1,  0,  4,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 18,
          0, 20, 21,  0,  0,  0,  0,  0, 27,  0]])


In [ ]:
embds1_by_hand = sdf_inference.decoder.Y(
    custom_mask1
)
embds1_by_hand = sdf_inference.decoder.activation(embds1_by_hand)
embds1_by_hand = embds1_by_hand.reshape(embds1_by_hand.shape[0], -1)
embds1_by_hand = sdf_inference.decoder.linear(embds1_by_hand)


embd_1, pooled_embd_1 = sdf_inference.overwrite_full_embedding(embds1_by_hand, idx=idx1)

image_generator.generate_image_from_embd(
    prompt_embeds=embd_1.detach(),
    pooled_prompt_embeds=pooled_embd_1.detach(),
    image_name="image_brune_street_insert_cola_reconstructed.png",
)

100%|██████████| 4/4 [00:03<00:00,  1.19it/s]


Image saved in image_brune_street_insert_cola_reconstructed.png
